<a href="https://colab.research.google.com/github/RubeRad/tcscs/blob/master/StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2

This notebook is adapted from [this notebook published by Mikael Christensen](https://t.co/hvB4OHthB5). For information on StyleGAN2, see:

* Paper: https://arxiv.org/abs/1812.04948
* Video: https://youtu.be/kSLJriaOumA
* Code: https://github.com/NVlabs/stylegan
* FFHQ: https://github.com/NVlabs/ffhq-dataset


# Setup Cells




There are a handful of cells which need to be run first, to get things ready.

If you come back to this notebook, you can collapse this setup section, and run all the hidden cells with one click

## Setup 1: Import tensorflow and clone StyleGAN2 code from github

After running this cell, check out what GPU device Google is letting your cloud computer use (for free)! Search for it on Amazon, how much does it cost to buy one?

Also, note in the Files sidebar, there is now a directory stylegan2 -- that is code pulled down from the git repository posted by NVIDIA at https://github.com/NVlabs/stylegan2.git

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

# avoid recursive cloning if this cell is re-run
# always start in top directory
%cd /content

# Download the StyleGAN2 code from github
!git clone https://github.com/NVlabs/stylegan2.git

# change into the repo directory, stay there for everything
%cd /content/stylegan2
!mkdir projection
!mkdir projection/target
!mkdir projection/out
!nvcc test_nvcc.cu -o test_nvcc -run

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

## Setup 2: Download pretrained model and import other modules

This imports a whole bunch more standard modules (StyleGAN2 is not a standard module, which is why it was cloned from github, not just imported).

Then it downloads one pretrained network; that's the setup of all the nodes, layers, weights, biases, activation function choices, etc. The comments list a large number of possible model choices, by default it chooses stylegan2-ffhq-config-f-pkl, which is for 1024x1024 images of faces.

In [ ]:
# Download the model of choice

import pretrained_networks

# Choose between these pretrained models - I think 'f' is the best choice:

# 1024×1024 faces
# stylegan2-ffhq-config-a.pkl
# stylegan2-ffhq-config-b.pkl
# stylegan2-ffhq-config-c.pkl
# stylegan2-ffhq-config-d.pkl
# stylegan2-ffhq-config-e.pkl
# stylegan2-ffhq-config-f.pkl

# 512×384 cars
# stylegan2-car-config-a.pkl
# stylegan2-car-config-b.pkl
# stylegan2-car-config-c.pkl
# stylegan2-car-config-d.pkl
# stylegan2-car-config-e.pkl
# stylegan2-car-config-f.pkl

# 256x256 horses
# stylegan2-horse-config-a.pkl
# stylegan2-horse-config-f.pkl

# 256x256 churches
# stylegan2-church-config-a.pkl
# stylegan2-church-config-f.pkl

# 256x256 cats
# stylegan2-cat-config-f.pkl
# stylegan2-cat-config-a.pkl
network_pkl = "gdrive:networks/stylegan2-ffhq-config-f.pkl"

# If downloads fails, due to 'Google Drive download quota exceeded' you can try downloading manually from your own Google Drive account
# network_pkl = "/content/drive/My Drive/GAN/stylegan2-ffhq-config-f.pkl"

print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

In [ ]:
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import pickle
import re
import sys
from io import BytesIO
import IPython.display
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio
import seaborn as sns

## Setup 3: Define a bunch of useful python functions

These were all from the original Mikael Christensen notebook

In [ ]:
# Useful utility functions...

# Generates a list of images, based on a list of latent vectors (Z), and a list (or a single constant) of truncation_psi's.
def generate_images_in_w_space(dlatents, truncation_psi=0.7):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    Gs_kwargs.truncation_psi = truncation_psi
    dlatent_avg = Gs.get_var('dlatent_avg') # [component]

    imgs = []
    for row, dlatent in log_progress(enumerate(dlatents), name = "Generating images"):
        #row_dlatents = (dlatent[np.newaxis] - dlatent_avg) * np.reshape(truncation_psi, [-1, 1, 1]) + dlatent_avg
        dl = (dlatent-dlatent_avg)*truncation_psi   + dlatent_avg
        row_images = Gs.components.synthesis.run(dlatent,  **Gs_kwargs)
        imgs.append(PIL.Image.fromarray(row_images[0], 'RGB'))
    return imgs       

def generate_images(zs, truncation_psi=0.7):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if not isinstance(truncation_psi, list):
        truncation_psi = [truncation_psi] * len(zs)
        
    imgs = []
    for z_idx, z in log_progress(enumerate(zs), size = len(zs), name = "Generating images"):
        Gs_kwargs.truncation_psi = truncation_psi[z_idx]
        noise_rnd = np.random.RandomState(1) # fix noise
        tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
        imgs.append(PIL.Image.fromarray(images[0], 'RGB'))
    return imgs

def generate_zs_from_seeds(seeds):
    zs = []
    for seed_idx, seed in enumerate(seeds):
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        zs.append(z)
    return zs

# Generates a list of images, based on a list of seed for latent vectors (Z), and a list (or a single constant) of truncation_psi's.
def generate_images_from_seeds(seeds, truncation_psi):
    return generate_images(generate_zs_from_seeds(seeds), truncation_psi)

def saveImgs(imgs, location):
  for idx, img in log_progress(enumerate(imgs), size = len(imgs), name="Saving images"):
    file = location+ str(idx) + ".png"
    img.save(file)

def imshow(a, format='png', jpeg_fallback=True):
  a = np.asarray(a, dtype=np.uint8)
  str_file = BytesIO()
  PIL.Image.fromarray(a).save(str_file, format)
  im_data = str_file.getvalue()
  try:
    disp = IPython.display.display(IPython.display.Image(im_data))
  except IOError:
    if jpeg_fallback and format != 'jpeg':
      print ('Warning: image was too large to display in format "{}"; '
             'trying jpeg instead.').format(format)
      return imshow(a, format='jpeg')
    else:
      raise
  return disp

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

        
def clamp(x, minimum, maximum):
    return max(minimum, min(x, maximum))
    
def drawLatent(image,latents,x,y,x2,y2, color=(255,0,0,100)):
  buffer = PIL.Image.new('RGBA', image.size, (0,0,0,0))
   
  draw = ImageDraw.Draw(buffer)
  cy = (y+y2)/2
  draw.rectangle([x,y,x2,y2],fill=(255,255,255,180), outline=(0,0,0,180))
  for i in range(len(latents)):
    mx = x + (x2-x)*(float(i)/len(latents))
    h = (y2-y)*latents[i]*0.1
    h = clamp(h,cy-y2,y2-cy)
    draw.line((mx,cy,mx,cy+h),fill=color)
  return PIL.Image.alpha_composite(image,buffer)
             
  
def createImageGrid(images, scale=0.25, rows=1):
   w,h = images[0].size
   w = int(w*scale)
   h = int(h*scale)
   height = rows*h
   cols = ceil(len(images) / rows)
   width = cols*w
   canvas = PIL.Image.new('RGBA', (width,height), 'white')
   for i,img in enumerate(images):
     img = img.resize((w,h), PIL.Image.ANTIALIAS)
     canvas.paste(img, (w*(i % cols), h*(i // cols))) 
   return canvas

def convertZtoW(latent, truncation_psi=0.7, truncation_cutoff=9):
  dlatent = Gs.components.mapping.run(latent, None) # [seed, layer, component]
  dlatent_avg = Gs.get_var('dlatent_avg') # [component]
  for i in range(truncation_cutoff):
    dlatent[0][i] = (dlatent[0][i]-dlatent_avg)*truncation_psi + dlatent_avg
    
  return dlatent

def generate_ws_from_zs(zs, truncation_psi=0.7, truncation_cutoff=9):
  ws = []
  for z in zs:
    ws.append( convertZtoW(z, truncation_psi, truncation_cutoff) )
  return ws

def interpolate(zs, steps):
   out = []
   for i in range(len(zs)-1):
    for index in range(steps):
     fraction = index/float(steps-1) 
     #print('interpolate {}/{}'.format(index,steps-1))
     out.append(zs[i+1]*fraction + zs[i]*(1-fraction))
   return out

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )


## Setup 4: Define some more python functions

These are functions I wrote, to make it more straightforward to do the kinds of things I'm wanting students to do.

Take a minute to look at the step-by-step working of `show_seeds()`

In [ ]:
def show_seeds(seeds):
  # Use each seed to initialize the rng and sample latent vectors:
  # 512 random variables from a Standard Normal distribution
  zs = generate_zs_from_seeds(seeds)

  # Map each latent vector into W-space that the generator accepts as input
  ws = generate_ws_from_zs(zs)

  # Push each W through the generator to yield an image
  images = generate_images_in_w_space(ws)

  # concatenate the images into a bigger gridded image
  grid = createImageGrid(images, scale=0.7, rows=3)

  # show them all!
  imshow(grid)

def seed2z(seed):
  zs = generate_zs_from_seeds( [seed] )
  return seeds[0]

def seeds2imgs(seeds):
  return generate_images_in_w_space( generate_ws_from_zs( generate_zs_from_seeds( seeds ) ) )

def seed2img(seed):
  imgs = seeds2imgs([seed])
  return imgs[0]

def show_seed_img(seed):
  imshow(seed2img(seed))

def w2img(wvec):
  imgs = generate_images_in_w_space( [wvec] )
  return imgs[0]

def z2img(zvec):
  ws = generate_ws_from_zs( [zvec] )
  return w2img(ws[0])

def show_z_img(latent):
  imshow(z2img(latent))

# For instance if you want to save a latent vector, dump(m_fave_z, 'myZ.pkl')
# Then DOWNLOAD a copy of it, or it will be recycled along with the 
# ephemeral cloud computer
def dump(thing, path):
  with open(path, 'wb') as file:
    pickle.dump(thing, file)

# In a later Colab session, upload your file and you can load it back in
# my_fave_z = load('myZ.pkl')
def load(path):
  with open(path, 'rb') as file:
    thing = pickle.load(file)
  return thing


# How a face is generated




This is a close up look of elements in creating a random face. Later, these steps will be wrapped in functions, or we might jump into the process in the middle.


## Generate a face: Step 1: choose a seed

Choose any random number

In [ ]:
seed = 1                              # first we'll all try this
# seed = np.random.randint(10000000)  # then we'll try different random seeds
seed

## Generate a Face: Step 2: seed the RNG

Random Number Generators are typically deterministic, and if you initialize them with the same seed, they will reliably generate the same stream of randomness.

In [ ]:
rng = np.random.RandomState(seed)

## Generate a Face Step 3: Randomly sample Z-space latent vector

The z-vector needs to be a row-vector of 512 samples (dimension 1x512) of a Standard Normal ($\mu=0$, $\sigma=1$) random variable. `randn()` is the numpy function for the Standard Normal distribution (rand"n" is for Normal)

In [ ]:
z = rng.randn(1, 512)
z

In [ ]:
z.shape

## Generate a Face Step 4: Convert latent vector from z-space to w-space

The StyleGAN2 Generator network is two components: mapping (z-to-w) and synthesis (w-to-image). This step is basically just inputting the z vector into the mapping network, and out pops w. But there are some other details we won't get into, wrapped up in the convertZtoW function.

The resulting w-space latent vector is shape (1,18,512). According to the StyleGAN2 paper, "the synthesis network _g_ consists of 18 layers -- two for each resolution ($4^2$ -- $1024^2$)" is probably where the 18 comes from.


In [ ]:
w = convertZtoW(z)
w


In [ ]:
w.shape

Take a look at the differences between the values in the z vector, and the values in the w-vector (and these histograms say nothing about which of the 512 dimensions all the numerical values are)

In [ ]:
# By definition, a Standard Normal Bell Curve
sns.histplot(z[0])

In [ ]:
sns.histplot(w[0,0])

## Generate a Face Step 5: Push the w-space vector through the synthesis network

This is basically just inputting the w vector into StyleGAN2's synthesis network, and then the pixels of the output image can be read off the nodes in the output layer. This is wrapped up in function `generate_images_in_w_space`

This function uses a progress bar, which is nice for later when generating multiple images

In [ ]:
ws = [w]                               # the function wants to be pased a list of w-space vectors...
imgs = generate_images_in_w_space(ws)  #...and return a list of images

## Generate a Face Step 6: display the image

The image display capability comes from `PIL` (python image library), again, we don't need to get into the details, it's wrapped up in function imshow

In [ ]:
img = imgs[0]  # we grab the first (i.e. one and only) image off the list
imshow(img)

## Generate a Face Step 7: generate directly from z

Function `convertZtoW()` does `Gs.component.mapping.run()` and function `generate_images_in_w_space()` does `Gs.component.synthesis.run()`, i.e. the two components of StyleGAN2 separately.

The function `generate_images()` does both in one shot, using the whole network as `Gs.run()`

In [ ]:
zimgs = generate_images([z]) # this also wants a list of z vectors, and returns a list of images
imshow(zimgs[0])

# Generate lots of random faces



Take a look at how function `show_seeds()` does all the steps above (as well as assembling a list of images into a grid image).

This cell uses chooses 9 random numbers and holds them in a list called `seeds`, and then shows them all with `show_seeds()`

Take note of the extremely wide variety of faces generated! Age, gender, skin color, pose, expression, hairstyles/hats, glasses/earrings/etc. Once in a while, though, it goofs and there are noticeable defects, which often appear as water droplets, or in extreme cases, globs of electric blue goo.

In [ ]:
# generate some random seeds
seeds = np.random.randint(10000000, size=9)

# print seeds in the same grid as the images
print(np.array(seeds).reshape(3,3))  

show_seeds(seeds)

# Z Interpolation



As seen above, an image can be generated starting from just a single number used to seed the rng, but what counts more is the latent vector z. 

It turns out, if you have any two latent vectors $z1$ and $z2$ (which generate two images img1 and img2), halfway between those vectors is an image which is 'halfway' between the images! Or $$z=\frac{1}{10}z1 + \frac{9}{10}z2$$ generates an image which is 10% of the way from the first image to the second, etc. 

A smooth path between the latent vectors, generates a smooth transition of generated images, which this next cell demonstrates.

In [ ]:
# Simple (Z) interpolation

# these are the two seeds Christensen left in the notebook
seeds = [5015289 , 9148088] # that hat tho!

# You can try out any other two numbers you want, or let this choose randomly:
#seeds = np.random.randint(10000000, size=2)

# print the seeds in case you see an interesting face; just jot down the 
# seed and you'll be able to regenerate it later!
print(seeds)

# This does the same thing above to rng two latent vectors 'z' using the seeds
z1, z2 = generate_zs_from_seeds(seeds)

# The interpolate() function scales linearly from z1 to z2, and this renders 
# the resulting sequence of images
number_of_steps = 9
number_of_rows  = 3
imgs = generate_images(interpolate([z1,z2], number_of_steps), 1.0)
imshow(createImageGrid(imgs, 0.4 , number_of_rows))
# If you want, you could change that from 9 steps, shown in rows of 3, to 
# 16 steps, shown in rows of 4, etc.

# Homework area



Insert code/text cells as necessary to do the homework here, which is: 

1. Run the 9-random-image generation a bunch of times, and make note of two seeds which generate faces which are different in many different ways (age, gender, etc). 
1. Use two code cells with `show_seed()` to generate your two chosen images by themselves.
1. Use another code cell to replicate the Z-interpolation code above and generate a **16-image** transition in a **4x4 grid**
1. Conclude with a text cell where you briefly discuss your image choice, and how well the Z-transition worked.

In [ ]:
# show your first image
seed1 = 

In [ ]:
# show your 2nd image
seed2 = 

In [ ]:
# generate a transition between them

Discuss...

# Mean Girl




If you sample all possible z-space vectors, and convert them into w-space vectors, and take the average, you get a 'mean face', which the paper says "is similar for all trained networks, and the interpolation towards it never seems to cause artifacts."

## Mean Girl Step 1: get the mean w-vector

The network (`Gs`) holds onto the mean w-vector, we just need to grab it.

In [ ]:
meanw1 = Gs.get_var('dlatent_avg')
meanw1.shape

## Mean Girl Step 2: copy 18 times

That vector is only 512 numbers. We need to stack that vector 18 times to be able to generate images with it.

In [ ]:
meanw = np.zeros(18*512).reshape(1,18,512) # this is the right shape, but all zeros
# copy the 512-vector into each of the 18 spots
for i in range(18):
  meanw[0][i] = meanw1
meanw.shape

## Mean Girl Step 2: who is the Mean Girl?

Let's take a look!

In [ ]:
imgs = generate_images_in_w_space( [meanw] )
imshow(imgs[0])

Hmmm, she doesn't look so mean after all!

**What does it mean**, to say that that is the "mean face" of StyleGAN2?

## Mean Girl Step 3: Pick a starting image

Choose a seed, either one you liked from before, or a random one, preferably somebody that look very different from the Mean Girl.

In [ ]:
seed = 24687
# seed = np.random.randint(10000000)
print('Current seed is {}'.format(seed))
zs = generate_zs_from_seeds( [seed] )
ws = generate_ws_from_zs( zs )
w1 = ws[0]
imgs = generate_images_in_w_space(ws)
imshow(imgs[0])

## Mean Girl Step 4: compute w-space vector for "Anti-Face"

Vectors (and matrices) are just piles of numbers, as long as any two vectors are the same shape, you can subtract them, term-by-term.

In [ ]:
dw = meanw - w1     # thus dw is the 'direction' from w1 to meanw
# meanw = w1 + dw   # this is just true, by rearranging the previous equation

# But what if, starting at w1 and moving towards meanw, 
# we don't just stop there, but keep going!
w2 = w1 + 2*dw


In [ ]:
# what does the anti-face look like?
imgs = generate_images_in_w_space( [w2] )
imshow(imgs[0])

## Mean Girl Step 5: display the Faces

In [ ]:
# This generates these three faces from their w-vectors
imgs = generate_images_in_w_space( [w1, meanw, w2] )

# scale=0.3 shrinks them so we can see together, and rows=1 will put them side-by-side
gridImg = createImageGrid(imgs, scale=0.3, rows=1)

imshow(gridImg)


## Mean Girl Step 6: Interpolate more smoothly between the face and anti-face

The `interpolate()` function which was used up above to smoothly scale between z-space vectors, also works for w-space vectors

In [ ]:
ws = interpolate( [w1, w2], 9 )
imgs = generate_images_in_w_space(ws)
gridImg = createImageGrid(imgs, scale=0.4, rows=3)
imshow(gridImg)

# Find Your Face!



StyleGAN2 comes with a projector that finds the closest generatable image based on any input image. This allows you to get a feeling for the diversity of the portrait manifold.

^That sentence is courtesy Mikael Christensen. What it means is, give it a picture of YOUR face (or maybe a celebrity you like) as a target, and it will search to find a w-space latent vector that generates an image as close as possible to that target. This is similar to iterative back-propagation, but adjusting the *inputs* rather than the *weights*.

## Find Your Face Step 1: Upload target image

Look over to the left, in the Files sidebar. Expand directories **stylegan2/projection/target/.** Drag the image onto the target directory. 

(If you don't see it right away, try collapsing and re-expanding the directory)


Only **ONE** image may be in the directory.

The image needs to be a **color .png**, with a size of exactly 1024x1024 pixels. The
image needs to have the eyes/mouth fairly-well aligned with the starting image 
for the search. A rule of thumb for that is to have about a finger's 
width of space below the chin, and a little bit of hair cut off the top. 

## Find Your Face Step 2: Find Your Face

Run the cell below, which uses to the projector to execute the search.

After a few seconds, you will see `0 / 1000`, and from there it takes about a second per step, so It will take maybe 10-15 minutes to complete.

Unfortunately, you can't just walk away, because if the Colab Notebook finishes and then times out, it will be recycled, and your output along with it.

While it's crunching, if you're curious, open up stylegan2/projection/out, and double-click on any image-stepNNNN.png to take a peek at its progress!

In [ ]:
# Convert uploaded images to TFRecords
import dataset_tool
dataset_tool.create_from_images("./projection/records/", "./projection/target/", True)

# Run the projector
import projector
import run_projector
import training.dataset
import training.misc
import os 

print('Loading images from "%s"...' % 'records')
dataset_obj = training.dataset.load_dataset(data_dir='projection', 
                                            tfrecord_dir='records', 
                                            max_label_size=0, 
                                            verbose=True, 
                                            repeat=False, 
                                            shuffle_mb=0)
assert dataset_obj.shape == Gs.output_shape[1:]


# Here we set up and use the projector
proj = projector.Projector()
proj.set_network(Gs)

# for a full run this should be 1000, but it can be set smaller for testing
proj.num_steps = 1000


# 1000 is a lot of images, and they don't change that fast
# only 'snapshot' (save out) every 10th one to put into the movie
num_snapshots = int(proj.num_steps/10)

images, _labels = dataset_obj.get_minibatch_np(1)
images = training.misc.adjust_dynamic_range(images, [0, 255], [-1, 1])
run_projector.project_image(proj, 
                            targets=images, 
                            png_prefix='projection/out/image-', 
                            num_snapshots=num_snapshots)

## Find Your Face: Step 3: Make a video

Once the finding is done, the directory stylegan2/projection/out/ is full of images named `image-step0NNNN.png`, all the "snapshots" it saved along the journey to finding your face.

This cell loads those image files and crunches them all together into a video, and saves out as a mp4 file. Each frame is the concatenation of the fixed target image on the left, and the current search step on the right. Note that the search starts at the Mean Face.

If you want the saved filename to be something different than `movie.mp4`, you can edit the `movieName` variable.

In [ ]:
# Create video 

import glob

imgs = sorted(glob.glob("projection/out/*step*.png"))

target_imgs = sorted(glob.glob("projection/out/*target*.png"))
assert len(target_imgs) == 1, "More than one target found?"
target_img = imageio.imread(target_imgs[0])

movieName = "projection/movie.mp4"

with imageio.get_writer(movieName, mode='I') as writer:
    for filename in log_progress(imgs, name = "Creating animation"):
        image = imageio.imread(filename)

        # Concatenate images with original target image
        w,h = image.shape[0:2]
        canvas = PIL.Image.new('RGBA', (w*2,h), 'white')
        canvas.paste(Image.fromarray(target_img), (0, 0))
        canvas.paste(Image.fromarray(image), (w, 0))

        writer.append_data(np.array(canvas))  

## Find Your Face: Step 4: Save your video

Go find your video file on the File sidebar on the left, click '...', and Download. If you didn't change the filename away from movie.mp4, you have
another opportunity to change the filename when you save the download.

## Find Your Face: Step 5a: Save your W-space vector

The next cell will save the W-space vector that the search finally settled on.
Later you can reload it and display it again. The StyleGAN2 code typically calls
this `dlatent` vs just `latent` for z

***IMPORTANT*** Download a copy of this file! If you don't download it and the Colab session times out, it will be gone!

In [ ]:
myW = proj.get_dlatents()

# You can change this filename if you want
filename = 'myWvector.pkl'

dump(myW, filename)

## Find Your Face: Step 5b: Reload your W-space vector

If you are returning to this notebook with a saved W-space vector, you can load it back in, instead of the long process to search for a near-match.

First upload your saved file to the colab session.

In [ ]:
# Change this filename to match what you uploaded
filename = 'myWvector.pkl'

myW = load(filename)

## Find Your Face: Step 6: Save your image

In addition to the side-by-side video from above, you can also save just an image.

In [ ]:
# This actually creates a list of images
imgs = generate_images_in_w_space([myW])
me = imgs[0]
imshow(me)

In [ ]:
# change this if you want
imgFilename = 'me.png'

me.save(imgFilename)

# Homework Area



Two options for homework:

1. Generate your anti-face. Start with `w1` as w for your face from the previous homework, and follow the steps above to compute `w2` for your anti-face, and generate a transition grid through the Mean Girl. (You will need to rerun some of the cells above to set `meanw` as well). **Discuss** what visible features of the result make that face the 'opposite' of yours. NOTE: a full 2 vectors away from yourself is probably TOO FAR beyond the Mean Girl on the other side. Scale down the factor of 2.0 until the end point still looks like a human being.

2. Morph yourself into a hero. Repeat the Find Your Face process to obtain a w-vector for the face of somebody you want to morph into, and create a transition grid (or if you're ambitius, video!) that morphs you into your hero! 
Let me know if you need help cropping/resampling your hero's face to a 1024x1024 png to start with. Discuss: did the transition go near the Mean Girl (or close)? Why/why not?

In [ ]:
# do the homework here, add more cells as necessary

Discuss...

# Boundary Vectors



In w-space, the vectors/points which generate images of various 'styles' (age, gender, etc) are well-separated into clusters, and we can use the vector between clusters to transition w-vectors forwards or backwards in that style

## Boundary Vectors Step 1: Upload vector data

Thanks to [github user a312863063](https://github.com/a312863063/generators-with-stylegan2/tree/master/latent_directions), there are lots of 'latent directions' to try out. Upload all the provided *.npy.txt files into the stylegan2/ directory

## Boundary Vectors Step 2: Choose a 'style'

Choose one of the *.npy.txt files and load it up.

In [ ]:
# Note as-is it's just a list of 18*512 numbers, 
# the reshape gets it ready for StyleGAN2
dw_age = np.loadtxt('age.npy.txt').reshape(1,18,512)

## Boundary Vectors Step 3: Choose a starting point

Use the techniques you should be good at by now to choose a w-space vector to start from.

In [ ]:
w = meanw

# or start with a w computed from a random seed, or loaded from a FYF


## Boundary Vectors Step 4: Modify the w-space vector

Move some distance backward (-dw) and forward (+dw) from the starting point. Depending on what style is being modified, and where you're starting from, you might move different distances back and forward. Note also the vectors are pretty short, so you might need to scale it up a lot.

In [ ]:
w1 = w + 2*dw_age
w2 = w - 2*dw_age

## Boundary Vectors Step 5: Create an interpolated transition grid

Now that we have endpoints in w-space, we can do the same interpolation as before.

In [ ]:
ws = interpolate( [w1, w2], 9)
imgs = generate_images_in_w_space(ws)
grid = createImageGrid(imgs, scale=0.3, rows=3)
imshow(grid)

# Homework Area



Using the techniques demonstrated above, make at least three transition grids that you like, using different 'style' dimensions. (For the starting face(s) you can use any w you want, and they don't have to be the same every time)

Don't adjust from the starting image so far forward/backward that endpoints look crazy (or at least comparably no crazier than the starting face)